In [6]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import codecs

stop_words = np.array(pd.read_csv('stopwords_russian2translit.txt', encoding='utf-8', lineterminator='\n')).ravel()

In [8]:
def parse_url(url, stop_words, min_len_word=1):
#   url?text#text -> url:
    temp = re.search(r'.+\?', url)
    if temp != None:
        url = temp.group(0)[:-1]
    temp = re.search(r'.+#', url)
    if temp != None:
        url = temp.group(0)[:-1]
    
#   site_name, full_domen, last_domen
    full_domen = re.search(r'\..+', url).group(0)[1:]
    if re.search(r'.+?/', full_domen) != None:
        full_domen = re.search(r'.+?/', full_domen).group(0)[:-1]    
    last_domen = full_domen.strip().split(sep='.')[-1]
    site_name = re.search(r'.+?\.', url).group(0)[:-1]
    
#   usefull_names = url.split() - domens - stop_words - set(word: len(word) < n)
    domens = full_domen.strip().split(sep='.')
    usefull_names = set(re.split('-|_|\.|/',url))
    usefull_names = usefull_names.difference(stop_words)
    usefull_names = usefull_names.difference(domens)
    usefull_names = [item for item in usefull_names if len(item) >= min_len_word]
    usefull_names = ' '.join(list(usefull_names))

        
    return full_domen, last_domen, site_name, usefull_names

In [107]:
output_txt = 'new_urls.txt'
path = 'content/'

with open(output_txt, 'w') as out_f:
    out_f.write('{}\t{}\n'.format('doc_id', 'usefull_names'))
    for doc_id in range(1, 28027):
        with codecs.open(path + str(doc_id) + '.dat', 'r', 'utf-8') as in_f:
            url = in_f.readline().strip()
            _, _, _, usefull_names = parse_url(url, stop_words, min_len_word=2)
            out_f.write('{}\t{}\n'.format(doc_id, usefull_names))

In [159]:
output_txt = 'ngrams.txt'
un = np.array(pd.read_csv('new_urls.txt', sep='\t', encoding='utf-8', lineterminator='\n').usefull_names).ravel()
# un = filter(lambda v: v==v, un)
n = 4

with open(output_txt, 'w') as out_f:
    out_f.write('{}\t{}\n'.format('doc_id', 'ngrams'))
    for doc_id, it in enumerate(un):
        if not isinstance(it, np.str):
            out_f.write(str(doc_id)+'\t'+''+'\n')    
            continue
        out_f.write(str(doc_id)+'\t')    
        ngr = ngrams(it, n)
        for grams in ngr:
            out_f.write(''.join(grams) + ',')
        out_f.write('\n')